# 3.4 3층 신경망 구축하기

## 3.4.3 구현 정리 - jupyter notebook에서 해보기

### forward propagation

In [1]:
# 출력층 신호 전달의 항등함수 정의
import numpy as np
def identity_function(x):
    return x

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [2]:
# 가중치와 편향의 np.array 만들기
def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])

    return network

In [3]:
# 순전파 계산하기
def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)

    return y

In [4]:
network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


# 3.5 출력층 설계하기

In [5]:
# softmax 함수 구현하기
# overflow 방지를 위해 np.max(a) - 입력 값 중 최대 값을 빼주어 계산하면 넘치지 않고 계산 가능
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # overflow 대체
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

# 3.6 손글씨 숫자 인식
기계 학습과 마찬가지로 신경망도 두 단계를 거쳐 문제를 해결한다.  
먼저 훈련 데이터를 사용해 가중치 매개변수를 학습하고,   
추론 단계에서는 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류한다.

글씨 mnist는 저자/역자가 제공한 mnist.py 이용해 다운로드

In [6]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

In [7]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [8]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [9]:
# 훈련이미지 시험삼아 보기
from PIL import Image

In [10]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) # uint 오타
    pil_img.show() # pil underbar img

In [11]:
img = x_train[0]
label = t_train[0]
print(label)

5


In [12]:
print(img.shape) # fromarry에 의해 28*28 = 784의 1차원 배열

(784,)


In [13]:
img = img.reshape(28, 28) # 다시 28*28의 2차원 행렬로
print(img.shape)

(28, 28)


In [14]:
img_show(img)

## 3.6.2 신경망의 추론 처리

In [16]:
# 이미 학습되어 있는 parameter가 저장되어 있는 pkl (pickle)파일 불러올 수 있는 모듈
import pickle

In [19]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [21]:
# prediction하고 accuracy 구하기
# 배치 처리로 더 빠르게
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy: " + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


컴퓨터에서 큰 배열을 한꺼번에 계산하는 것이 분할된 작은 배열을 여러 번 계산하는 것보다 빠르다